In [3]:
import pandas as pd

df_map = pd.read_csv('https://raw.githubusercontent.com/LaraLim/cnv-supervised-learning/refs/heads/main/sample_mapping.csv?token=GHSAT0AAAAAACXNJQCFXPY5VH5FEFSBG5NCZY67QXA')


combined_df = pd.DataFrame()
combined_df['CaseID'] = ''

#first sample 
filename = df_map['Filename'][0]

#get the shape of the dataframe
url = f'https://raw.githubusercontent.com/LaraLim/cnv-supervised-learning/refs/heads/main/gene_level_cnv/{filename}'
df = pd.read_csv(url, sep='\t')
numRows = df.shape[0]

#dictionary of dataframes for each id 
dfs = {}

print(f'{filename} has {df.shape[0]} rows')


for filename in df_map['Filename']:

    #append new column for the case id
    case_id = df_map[df_map['Filename'] == filename]['CaseID'].values[0]


    url = f'https://raw.githubusercontent.com/LaraLim/cnv-supervised-learning/refs/heads/main/gene_level_cnv/{filename}'
    df = pd.read_csv(url, sep='\t')

    #check if the number of rows is the same
    if df.shape[0] != numRows: 
        print("The number of rows is not the same between all files")
    
    df['CaseID'] = case_id
    df['SumGeneExpression'] = 0
    df['MeanGeneExpression'] = 0
    df['VarianceGeneExpression'] = 0

    df['status'] = 'normal'
    df.loc[df['copy_number'] > 2, 'status'] = 'amplified'
    df.loc[df['copy_number'] < 2, 'status'] = 'deleted'

    #move the position of the copy number column to the last column
    copy_number = df.pop('copy_number')
    df['copy_number_target'] = copy_number

    
    #add the dataframe to the dictionary
    dfs[case_id] = df
    
    combined_df = pd.concat([combined_df, df], ignore_index=True)

   
display(combined_df.head())

#display the bottom of the dataframe
#display(combined_df.tail())


ee5e869c-e15f-4899-9e12-377920609b42.wgs.ASCAT.gene_level.copy_number_variation.tsv has 60623 rows


,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,SumGeneExpression,MeanGeneExpression,VarianceGeneExpression,status,copy_number_target
0,C3L-00359-01,ENSG00000223972.5,DDX11L1,chr1,11869.0,14409.0,4.0,4.0,0.0,0.0,0.0,amplified,4.0
1,C3L-00359-01,ENSG00000227232.5,WASH7P,chr1,14404.0,29570.0,4.0,4.0,0.0,0.0,0.0,amplified,4.0
2,C3L-00359-01,ENSG00000278267.1,MIR6859-1,chr1,17369.0,17436.0,4.0,4.0,0.0,0.0,0.0,amplified,4.0
3,C3L-00359-01,ENSG00000243485.5,MIR1302-2HG,chr1,29554.0,31109.0,4.0,4.0,0.0,0.0,0.0,amplified,4.0
4,C3L-00359-01,ENSG00000284332.1,MIR1302-2,chr1,30366.0,30503.0,4.0,4.0,0.0,0.0,0.0,amplified,4.0


In [4]:
for case_id, df in dfs.items():
    display(df.head())

,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,CaseID,SumGeneExpression,MeanGeneExpression,VarianceGeneExpression,status,copy_number_target
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,4.0,4.0,C3L-00359-01,0,0,0,amplified,4.0
1,ENSG00000227232.5,WASH7P,chr1,14404,29570,4.0,4.0,C3L-00359-01,0,0,0,amplified,4.0
2,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,4.0,4.0,C3L-00359-01,0,0,0,amplified,4.0
3,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,4.0,4.0,C3L-00359-01,0,0,0,amplified,4.0
4,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,4.0,4.0,C3L-00359-01,0,0,0,amplified,4.0


,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,CaseID,SumGeneExpression,MeanGeneExpression,VarianceGeneExpression,status,copy_number_target
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,4.0,4.0,C3L-00606-01,0,0,0,amplified,4.0
1,ENSG00000227232.5,WASH7P,chr1,14404,29570,4.0,4.0,C3L-00606-01,0,0,0,amplified,4.0
2,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,4.0,4.0,C3L-00606-01,0,0,0,amplified,4.0
3,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,4.0,4.0,C3L-00606-01,0,0,0,amplified,4.0
4,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,4.0,4.0,C3L-00606-01,0,0,0,amplified,4.0


,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,CaseID,SumGeneExpression,MeanGeneExpression,VarianceGeneExpression,status,copy_number_target
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,5.0,5.0,C3L-00606-02,0,0,0,amplified,5.0
1,ENSG00000227232.5,WASH7P,chr1,14404,29570,5.0,5.0,C3L-00606-02,0,0,0,amplified,5.0
2,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,5.0,5.0,C3L-00606-02,0,0,0,amplified,5.0
3,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,5.0,5.0,C3L-00606-02,0,0,0,amplified,5.0
4,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,5.0,5.0,C3L-00606-02,0,0,0,amplified,5.0


,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,CaseID,SumGeneExpression,MeanGeneExpression,VarianceGeneExpression,status,copy_number_target
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,4.0,4.0,C3L-00606-03,0,0,0,amplified,4.0
1,ENSG00000227232.5,WASH7P,chr1,14404,29570,4.0,4.0,C3L-00606-03,0,0,0,amplified,4.0
2,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,4.0,4.0,C3L-00606-03,0,0,0,amplified,4.0
3,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,4.0,4.0,C3L-00606-03,0,0,0,amplified,4.0
4,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,4.0,4.0,C3L-00606-03,0,0,0,amplified,4.0


,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,CaseID,SumGeneExpression,MeanGeneExpression,VarianceGeneExpression,status,copy_number_target
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,1.0,1.0,C3L-01287-01,0,0,0,deleted,1.0
1,ENSG00000227232.5,WASH7P,chr1,14404,29570,1.0,1.0,C3L-01287-01,0,0,0,deleted,1.0
2,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,1.0,1.0,C3L-01287-01,0,0,0,deleted,1.0
3,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,1.0,1.0,C3L-01287-01,0,0,0,deleted,1.0
4,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,1.0,1.0,C3L-01287-01,0,0,0,deleted,1.0


...

,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,CaseID,SumGeneExpression,MeanGeneExpression,VarianceGeneExpression,status,copy_number_target
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,2.0,2.0,C3N-02783-05,0,0,0,normal,2.0
1,ENSG00000227232.5,WASH7P,chr1,14404,29570,2.0,2.0,C3N-02783-05,0,0,0,normal,2.0
2,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,2.0,2.0,C3N-02783-05,0,0,0,normal,2.0
3,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,2.0,2.0,C3N-02783-05,0,0,0,normal,2.0
4,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,2.0,2.0,C3N-02783-05,0,0,0,normal,2.0


,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,CaseID,SumGeneExpression,MeanGeneExpression,VarianceGeneExpression,status,copy_number_target
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,2.0,2.0,C3N-02784-01-03,0,0,0,normal,2.0
1,ENSG00000227232.5,WASH7P,chr1,14404,29570,2.0,2.0,C3N-02784-01-03,0,0,0,normal,2.0
2,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,2.0,2.0,C3N-02784-01-03,0,0,0,normal,2.0
3,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,2.0,2.0,C3N-02784-01-03,0,0,0,normal,2.0
4,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,2.0,2.0,C3N-02784-01-03,0,0,0,normal,2.0


,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,CaseID,SumGeneExpression,MeanGeneExpression,VarianceGeneExpression,status,copy_number_target
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,7.0,7.0,C3N-03184-02,0,0,0,amplified,7.0
1,ENSG00000227232.5,WASH7P,chr1,14404,29570,7.0,7.0,C3N-03184-02,0,0,0,amplified,7.0
2,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,7.0,7.0,C3N-03184-02,0,0,0,amplified,7.0
3,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,7.0,7.0,C3N-03184-02,0,0,0,amplified,7.0
4,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,7.0,7.0,C3N-03184-02,0,0,0,amplified,7.0


,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,CaseID,SumGeneExpression,MeanGeneExpression,VarianceGeneExpression,status,copy_number_target
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,2.0,2.0,C3N-03186-01,0,0,0,normal,2.0
1,ENSG00000227232.5,WASH7P,chr1,14404,29570,2.0,2.0,C3N-03186-01,0,0,0,normal,2.0
2,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,2.0,2.0,C3N-03186-01,0,0,0,normal,2.0
3,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,2.0,2.0,C3N-03186-01,0,0,0,normal,2.0
4,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,2.0,2.0,C3N-03186-01,0,0,0,normal,2.0


,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,CaseID,SumGeneExpression,MeanGeneExpression,VarianceGeneExpression,status,copy_number_target
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,5.0,5.0,C3N-03188-02,0,0,0,amplified,5.0
1,ENSG00000227232.5,WASH7P,chr1,14404,29570,5.0,5.0,C3N-03188-02,0,0,0,amplified,5.0
2,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,5.0,5.0,C3N-03188-02,0,0,0,amplified,5.0
3,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,5.0,5.0,C3N-03188-02,0,0,0,amplified,5.0
4,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,5.0,5.0,C3N-03188-02,0,0,0,amplified,5.0
